In [0]:
from rootpy.plotting import Hist, HistStack, Canvas, Legend
from ROOT import TPaveText
from numpy import sqrt
from mut_framework.BTagSF_FtCM.DataManager import Sample, DataManager

kappa = 1.00
ttbar_xs = 240.0
mc_samples = [Sample("../output/TTbar_Summer13.json", "TTbar", 6923750,ttbar_xs, isSignal = True),
           Sample("../output/WJets_Summer13.json", "WJets", 76041475, kappa*36257.2),
           Sample("../output/T_tW_Summer13.json", "T_tW", 497658, kappa*11.15),
           Sample("../output/Tbar_tW_Summer13.json", "Tbar_tW", 493460, kappa*11.15),
           Sample("../output/Tbar_t_Summer13.json", "Tbar_t", 1935072, kappa*30.0),
           Sample("../output/ZJets_Summer13.json", "ZJets", 30459503, kappa*3503.0),
           Sample("../output/T_t_Summer13.json", "T_t", 3758227, kappa*55.531)]
data_samples = [Sample("../output/Data_2012ABCD_Winter13_ReReco.json", "Data_2012ABCD_Winter13_ReReco", 0.0, 0.0)]

lumi = 19789.0

dm = DataManager(mc_samples, data_samples, lumi )
dm.set_tag_wp("combinedSecondaryVertexBJetTags", 0.244)

In [0]:

c = Canvas(1000,800)

ptBins = dm.ptBins()
good_jets = dm.good_jets_by_type()
h_format = {"data" : [None, "black", 0, "Data"],
            "b_jets": ["solid","red", 0, "b-flavour jets"],
            "c_jets": ["solid","green", 0, "c-flavour jets"],
            "l_jets": ["solid","blue", 0, "light jets" ]}
h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin= 0.55)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(28)

for k,v in good_jets.items():
    h_dict[k] = Hist( ptBins, title = h_format[k][3])
    if k == "data":
        for i in range(h_dict[k].nbins()):
            h_dict[k][i+1] = (v[i], sqrt(v[i]))
    else:
        h_dict[k][1:-1] = v
    h_dict[k].fillstyle = h_format[k][0]
    h_dict[k].fillcolor = h_format[k][1]
    h_dict[k].linecolor = h_format[k][1]
    h_dict[k].linewidth = h_format[k][2]

for j_cat in ["b_jets", "c_jets", "l_jets"]:    
    h_stack.Add(h_dict[j_cat])
    legend.AddEntry(h_dict[j_cat], style = "F")
    
h_stack.Draw("'HIST E1 X0")
h_stack.SetTitle("Pre-tagging p_{t} spectrum - Jet Flavour composition")
h_stack.GetXaxis().SetTitle("Jet p_{T}[GeV/c]")
h_stack.GetYaxis().SetTitle("# good jets")
h_stack.GetXaxis().SetMoreLogLabels()
h_stack.GetXaxis().SetNoExponent()
h_dict["data"].Draw("SAME E1")
legend.AddEntry(h_dict["data"], style='LEP')
legend.Draw()
c.SetLogx(1)
c

In [0]:

c = Canvas(1000,800)

ptBins = dm.ptBins()
good_jets = dm.good_jets_by_sample()
h_format = {"Data" : [["data"], None, "black", 0],
            "t#bar{t}": [["TTbar"],"solid","red", 0],
            "Single top": [["T_t","Tbar_t"],"solid","green", 0],
            "t+W": [["T_tW","Tbar_tW"],"solid","yellow", 0 ],
            "Z+jets": [["ZJets"],"solid","orange", 0],
            "W+jets": [["WJets"],"solid","blue",0,"W+jets"]}
h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin= 0.55)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(28)
for k,v in h_format.items():
    h_dict[k] = Hist( ptBins, title = k)
    if k == "Data":
        for i in range(h_dict[k].nbins()):
            count = good_jets["data"][i]
            h_dict[k][i+1] = (count, sqrt(count))
    else:
        counts = [sum(c_t) for c_t in zip(*[good_jets[key] for key in v[0]])]
        h_dict[k][1:-1] = counts
        h_dict[k].fillstyle = h_format[k][1]
        h_dict[k].fillcolor = h_format[k][2]
        h_dict[k].linecolor = h_format[k][2]
        h_dict[k].linewidth = h_format[k][3]

for s in ["Single top", "t+W","Z+jets","W+jets", "t#bar{t}"]:
    h_stack.Add(h_dict[s])
    legend.AddEntry(h_dict[s], style = "F")
    
h_stack.Draw("'HIST E1 X0")
h_stack.SetTitle("Pre-tagging p_{t} spectrum - MC composition")
h_stack.GetXaxis().SetTitle("Jet p_{T}[GeV/c]")
h_stack.GetYaxis().SetTitle("# good jets")
h_stack.GetXaxis().SetMoreLogLabels()
h_stack.GetXaxis().SetNoExponent()
h_dict["Data"].Draw("SAME E1")
legend.AddEntry(h_dict["Data"], style='LEP')
legend.Draw()
c.SetLogx(1)


c_bias = Canvas(1000,500)


c_bins = [20, 30, 40, 50, 60, 70, 80,100, 120, 160, 210, 260, 320, 400, 500, 600, 800]
bins = [30, 50, 80, 120, 160, 320]

c_hist = Hist(c_bins)
c_hist.SetStats(False)
c_hist.SetAxisRange(0.7,1.3,"Y")
c_hist.GetXaxis().SetMoreLogLabels()
c_hist.GetXaxis().SetNoExponent()
c_hist.SetXTitle("Jet p_{T}[GeV/c]")
c_hist.SetYTitle("Data/Sim. pre-tag SF")
c_hist.SetTitle("CMS prelim. 20 fb^{-1} at #sqrt{s} = 8 TeV ")
c_hist.Draw()


hist = Hist( bins, title = "FtCM")
for i in range(1,len(bins)):
    print i
    data_val = h_dict["Data"].GetBinContent(i)
    data_err = sqrt(data_val)
    mc_val = h_stack.GetStack().Last().GetBinContent(i)
    sf = data_val/mc_val
    print sf
    sf_err = data_err/mc_val
    hist[i] = (sf,sf_err)
hist.Draw("SAME E1")


text_box = TPaveText(0.60,0.7,0.87,0.87, "NDC")
text_box.AddText("pt-dependent FtCM")
text_box.AddText(" - #sigma_{t#bar{t}}" + 
                 " = {:4.0f} ".format(ttbar_xs) +
                 " fb^{-1}")
text_box.AddText(" - #kappa" + 
                 " = {:5.2f}".format(kappa ))
text_box.Draw()

c_bias.SetLogx(1)
c_bias.GetPad(0).SetTickx()
c_bias.GetPad(0).SetTicky()
t = c.GetPrimitive("title")
t.SetTextSize(0.04)
t.SetBBoxCenterX(300)
t.SetBBoxCenterY(30)


c


In [0]:

c = Canvas()

ptBins = dm.ptBins()
tag_jets = dm.tag_jets_by_type()
h_format = {"data" : [None, "black", 0],
            "b_jets": ["solid","red", 0],
            "c_jets": ["solid","green", 0],
            "l_jets": ["solid","blue", 0 ]}
h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin=0.45, margin=0.3)
for k,v in tag_jets.items():
    h_dict[k] = Hist( ptBins, title = k)
    if k == "data":
        for i in range(h_dict[k].nbins()):
            h_dict[k][i+1] = (v[i], sqrt(v[i]))
    else:
        h_dict[k][1:-1] = v
    h_dict[k].fillstyle = h_format[k][0]
    h_dict[k].fillcolor = h_format[k][1]
    h_dict[k].linecolor = h_format[k][1]
    h_dict[k].linewidth = h_format[k][2]

for j_cat in ["b_jets", "c_jets", "l_jets"]:    
    h_stack.Add(h_dict[j_cat])
    legend.AddEntry(h_dict[j_cat], style = "F")
    
h_stack.Draw("'HIST E1 X0")
h_dict["data"].Draw("SAME E1 X0")
legend.AddEntry(h_dict["data"], style='LEP')
legend.Draw()
c.SetLogx(1)

c

In [0]:

c = Canvas(1000,800)

ptBins = dm.ptBins()
tag_jets = dm.tag_jets_by_sample()
h_format = {"Data" : [["data"], None, "black", 0],
            "t#bar{t}": [["TTbar"],"solid","red", 0],
            "Single top": [["T_t","Tbar_t"],"solid","green", 0],
            "t+W": [["T_tW","Tbar_tW"],"solid","yellow", 0 ],
            "Z+jets": [["ZJets"],"solid","orange", 0],
            "W+jets": [["WJets"],"solid","blue",0,"W+jets"]}
h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin= 0.55)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(28)
for k,v in h_format.items():
    h_dict[k] = Hist( ptBins, title = k)
    if k == "Data":
        for i in range(h_dict[k].nbins()):
            count = tag_jets["data"][i]
            h_dict[k][i+1] = (count, sqrt(count))
    else:
        counts = [sum(c_t) for c_t in zip(*[tag_jets[key] for key in v[0]])]
        h_dict[k][1:-1] = counts
        h_dict[k].fillstyle = h_format[k][1]
        h_dict[k].fillcolor = h_format[k][2]
        h_dict[k].linecolor = h_format[k][2]
        h_dict[k].linewidth = h_format[k][3]

for s in ["Single top", "t+W","Z+jets","W+jets", "t#bar{t}"]:
    h_stack.Add(h_dict[s])
    legend.AddEntry(h_dict[s], style = "F")
    
h_stack.Draw("'HIST E1 X0")
h_stack.SetTitle("After-tagging p_{t} spectrum - MC composition")
h_stack.GetXaxis().SetTitle("Jet p_{T}[GeV/c]")
h_stack.GetYaxis().SetTitle("# good jets")
h_stack.GetXaxis().SetMoreLogLabels()
h_stack.GetXaxis().SetNoExponent()
h_dict["Data"].Draw("SAME E1")
legend.AddEntry(h_dict["Data"], style='LEP')
legend.Draw()
c.SetLogx(1)

c

In [0]:
c = Canvas(1000,800)

tag_mul = dm.tag_mul_by_sample()
tmBins = range(len(tag_mul["data"])+1)
h_format = {"Data" : [["data"], None, "black", 0],
            "t#bar{t}": [["TTbar"],"solid","red", 0],
            "Single top": [["T_t","Tbar_t"],"solid","green", 0],
            "t+W": [["T_tW","Tbar_tW"],"solid","yellow", 0 ],
            "Z+jets": [["ZJets"],"solid","orange", 0],
            "W+jets": [["WJets"],"solid","blue",0,"W+jets"]}
h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin= 0.55)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(28)
for k,v in h_format.items():
    h_dict[k] = Hist( tmBins, title = k)
    if k == "Data":
        for i in range(h_dict[k].nbins()):
            count = tag_mul["data"][i]
            h_dict[k][i+1] = (count, sqrt(count))
    else:
        counts = [sum(c_t) for c_t in zip(*[tag_mul[key] for key in v[0]])]
        h_dict[k][1:-1] = counts
        h_dict[k].fillstyle = h_format[k][1]
        h_dict[k].fillcolor = h_format[k][2]
        h_dict[k].linecolor = h_format[k][2]
        h_dict[k].linewidth = h_format[k][3]

for s in ["Single top", "t+W","Z+jets","W+jets", "t#bar{t}"]:
    h_stack.Add(h_dict[s])
    legend.AddEntry(h_dict[s], style = "F")
    
h_stack.Draw("'HIST E1 X0")
h_stack.SetTitle("Tag-jet Multiplicity - MC composition")
h_stack.GetXaxis().SetTitle("Tag-jet Multiplicity")
h_stack.GetYaxis().SetTitle("# events")
h_dict["Data"].Draw("SAME E1")
legend.AddEntry(h_dict["Data"], style='LEP')
legend.Draw()

c

In [0]:
c = Canvas(1000, 800)

jet_mul = dm.jet_mul_by_sample()
tmBins = range(len(jet_mul["data"])+1)
h_format = {"Data" : [["data"], None, "black", 0],
            "t#bar{t}": [["TTbar"],"solid","red", 0],
            "Single top": [["T_t","Tbar_t"],"solid","green", 0],
            "t+W": [["T_tW","Tbar_tW"],"solid","yellow", 0 ],
            "Z+jets": [["ZJets"],"solid","orange", 0],
            "W+jets": [["WJets"],"solid","blue",0,"W+jets"]}
h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin= 0.55)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(28)
for k,v in h_format.items():
    h_dict[k] = Hist( tmBins, title = k)
    if k == "Data":
        for i in range(h_dict[k].nbins()):
            count = jet_mul["data"][i]
            h_dict[k][i+1] = (count, sqrt(count))
    else:
        counts = [sum(c_t) for c_t in zip(*[jet_mul[key] for key in v[0]])]
        h_dict[k][1:-1] = counts
        h_dict[k].fillstyle = h_format[k][1]
        h_dict[k].fillcolor = h_format[k][2]
        h_dict[k].linecolor = h_format[k][2]
        h_dict[k].linewidth = h_format[k][3]

for s in ["Single top", "t+W","Z+jets","W+jets", "t#bar{t}"]:
    h_stack.Add(h_dict[s])
    legend.AddEntry(h_dict[s], style = "F")
    
h_stack.Draw("'HIST E1 X0")
h_stack.SetTitle("Jet Multiplicity - MC composition")
h_stack.GetXaxis().SetTitle("Jet Multiplicity")
h_stack.GetYaxis().SetTitle("# events")
h_dict["Data"].Draw("SAME E1")
legend.AddEntry(h_dict["Data"], style='LEP')
legend.Draw()

c